In [1]:
import pandas as pd
train=pd.read_csv('train.csv')
train

,ID,Filename,Label,ClassName
0,16257,AnnualCrop/AnnualCrop_142.jpg,0,AnnualCrop
1,3297,HerbaceousVegetation/HerbaceousVegetation_2835...,2,HerbaceousVegetation
2,17881,PermanentCrop/PermanentCrop_1073.jpg,6,PermanentCrop
3,2223,Industrial/Industrial_453.jpg,4,Industrial
4,4887,HerbaceousVegetation/HerbaceousVegetation_1810...,2,HerbaceousVegetation
...,...,...,...,...
18895,4498,HerbaceousVegetation/HerbaceousVegetation_1952...,2,HerbaceousVegetation
18896,1149,Pasture/Pasture_1252.jpg,5,Pasture
18897,15489,AnnualCrop/AnnualCrop_2332.jpg,0,AnnualCrop
18898,6287,Residential/Residential_332.jpg,7,Residential


In [2]:
num_classes=train['ClassName'].nunique()
print(num_classes)

10


In [3]:
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from torchvision import transforms
class CustomDataset(Dataset):
    def __init__(self,path,transform):
        self.data=pd.read_csv(path)
        self.transform=transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img_path=self.data.iloc[index,1]
        image=Image.open(img_path).convert('RGB')
        label=self.data.iloc[index,2]
        classname=self.data.iloc[index,3]
        
        if self.transform:
            image=self.transform(image)
            
        return image,label,classname
    
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

path='train.csv'
train=CustomDataset(path,transform)

c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_py

In [4]:
train_loader=DataLoader(train,batch_size=64,shuffle=True)
train_loader

In [5]:
path='validation.csv'
val=CustomDataset(path,transform)
val_loader=DataLoader(train,batch_size=32,shuffle=True)

path='test.csv'
test=CustomDataset(path,transform)
test_loader=DataLoader(test,batch_size=64,shuffle=False)

In [9]:
from torchvision.models import vgg16
import torch.nn as nn
base_model = vgg16(pretrained=True)
num_features = base_model.classifier[6].in_features
base_model.classifier = nn.Sequential(
    *list(base_model.classifier.children())[:-1]
)
print(base_model)

c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import numpy as np
import GPyOpt
import pandas as pd
from itertools import product
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'


def get_model(num_layers, num_neurons, dropout, activation, weight_initializer, num_classes):
    base_model = models.vgg16(pretrained=True)
    for param in base_model.parameters():
        param.requires_grad = False
    num_features = base_model.classifier[6].in_features
    base_model.classifier = nn.Sequential(
        *list(base_model.classifier.children())[:-1]
    )
    layers = []
    in_features = num_features
    for i in range(num_layers):
        layers.append(nn.Linear(in_features, num_neurons[i]))
        layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Dropout(dropout[i]))
        layers.append(nn.BatchNorm1d(num_neurons[i]))
        in_features = num_neurons[i]
    layers.append(nn.Linear(in_features, num_classes))
    layers.append(nn.Softmax(dim=1))
    model = nn.Sequential(
        base_model,
        *layers
    )
    
    return model

try:
    log_df = pd.read_csv('AutoFCL_results.csv', header=0, index_col=['index'])
except FileNotFoundError:
    log_df = pd.DataFrame(columns=['index', 'activation', 'weight_initializer', 'dropout', 'num_neurons', 'num_layers', 'train_loss', 'val_loss'])
    log_df = log_df.set_index('index')


# Define hyperparameter search space
p_space = {
    'activation': ['relu', 'tanh', 'sigmoid'], 
    'weight_initializer': ['he_normal'], 
    'num_layers': list(range(0, 2)) 
}

p_space = list(product(*p_space.values()))

# Loop over the hyperparameter combinations and conduct Bayesian optimization
val_loss_min=np.Inf
for combo in p_space:
    activation, weight_initializer, num_layers = combo
    bounds = [] 
    for i in range(num_layers): 
        bounds.append({'name': 'dropout' + str(i + 1), 'type': 'discrete', 'domain': [0, 0.1, 0.2, 0.3, 0.4, 0.5]})
    for i in range(num_layers): 
        bounds.append({'name': 'num_neurons' + str(i + 1), 'type': 'discrete', 'domain': [2 ** j for j in range(6, 11)]})

    def train_model(x):
        # Extract dropout rates and number of neurons from the optimization parameters
        dropouts = [float(x[:, i]) for i in range(0, num_layers)]
        neurons = [int(x[:, i]) for i in range(num_layers, len(bounds))]
        
        # Create the model with the current set of hyperparameters
        model = get_model(
            num_layers=num_layers,
            num_neurons=neurons,
            dropout=dropouts,
            activation=activation,
            weight_initializer=weight_initializer,
            num_classes=num_classes
        )
        
        # Define the optimizer and criterion
        optimizer = optim.Adagrad(model.parameters())
        criterion = nn.CrossEntropyLoss()
        num_epochs=50
        
        # Train the model
        train_loss = 0.0
        for epoch in range(tqdm(range(num_epochs))):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
        
        # Compute validation loss
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item() * inputs.size(0)
            train_loss = train_loss/len(train_loader.dataset)
            val_loss = val_loss/len(val_loader.dataset)
            
            print(f'\nTraining Loss: {train_loss:.4f} \tValidation Loss: {val_loss:.4f}')
            if val_loss <= val_loss_min:
                torch.save(model.state_dict(), 'resnet101.pt')
                val_loss_min = val_loss
                
        log_df.loc[len(log_df)] = [activation, weight_initializer, dropouts, neurons, num_layers, train_loss, val_loss]
        
        return val_loss
    
    # Create an instance of Bayesian optimization
    opt_ = GPyOpt.methods.BayesianOptimization(f=train_model, domain=bounds)
    
    # Run the optimization
    opt_.run_optimization(max_iter=5)
    
    # Save results to CSV
    log_df.to_csv('AutoFCL_results.csv')

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
import torch
from sklearn.metrics import classification_report

def test(model, test_loader, criterion, device, num_classes):
    model.eval()
    class_correct = [0.0] * num_classes
    class_total = [0.0] * num_classes
    test_loss = 0.0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            test_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(output, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += (predicted[i] == labels[i]).item()
                class_total[label] += 1
    
    test_loss /= len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')

    for i in range(num_classes):
        if class_total[i] > 0:
            print(f'Accuracy of class {i}: {100 * class_correct[i] / class_total[i]:.2f}%')
    
    overall_accuracy = 100 * correct / total
    print(f'Overall Accuracy: {overall_accuracy:.2f}%')
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))